In [3]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf,:
    print(module.__name__, module.__version__)


2.15.0
sys.version_info(major=3, minor=10, micro=13, releaselevel='final', serial=0)
matplotlib 3.7.4
numpy 1.26.3
pandas 2.2.0
sklearn 1.2.2
tensorflow 2.15.0


In [6]:
import tensorflow_datasets as tfds

dataset,info = tfds.load('imdb_reviews/subwords8k',with_info = True,
                        as_supervised = True
                        )
train_dataset,test_dataset = dataset['train'],dataset['test']

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteH0GYP2/imdb_reviews-train.tfrecord…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteH0GYP2/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteH0GYP2/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0. Subsequent calls will reuse this data.


In [7]:
#通过encoder把我们的词语转换为subword形式

tokenizer = info.features['text'].encoder

print('vocabulary size: {}'.format(tokenizer.vocab_size))

vocabulary size: 8185


In [9]:
for i in train_dataset.take(1):
    print(np.array(i[0]))
    print(tokenizer.decode(np.array(i[0])))

[  62   18   41  604  927   65    3  644 7968   21   35 5096   36   11
   43 2948 5240  102   50  681 7862 1244    3 3266   29  122  640    2
   26   14  279  438   35   79  349  384   11 1991    3  492   79  122
  188  117   33 4047 4531   14   65 7968    8 1819 3947    3   62   27
    9   41  577 5044 2629 2552 7193 7961 3642    3   19  107 3903  225
   85  198   72    1 1512  738 2347  102 6245    8   85  308   79 6936
 7961   23 4981 8044    3 6429 7961 1141 1335 1848 4848   55 3601 4217
 8050    2    5   59 3831 1484 8040 7974  174 5773   22 5240  102   18
  247   26    4 3903 1612 3902  291   11    4   27   13   18 4092 4008
 7961    6  119  213 2774    3   12  258 2306   13   91   29  171   52
  229    2 1245 5790  995 7968    8   52 2948 5240 8039 7968    8   74
 1249    3   12  117 2438 1369  192   39 7975]
This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in histo

In [10]:
sample_string = 'Tensorflow is cool.'

tokenized_string = tokenizer.encode(sample_string)
# tokenize通过encoder把我们的词语转换为subword形式
print('Tokenized string is {}'.format(tokenized_string))
print(type(tokenized_string))

original_string = tokenizer.decode(tokenized_string)
print('Original string is {}'.format(original_string))

assert original_string == sample_string

Tokenized string is [6307, 2327, 2934, 7961, 9, 2724, 7975]
<class 'list'>
Original string is Tensorflow is cool.


In [13]:
for token in tokenized_string:
    print('{} -->{}'.format(token,tokenizer.decode([token])))

6307 -->Ten
2327 -->sor
2934 -->flow
7961 --> 
9 -->is 
2724 -->cool
7975 -->.


In [20]:
buffer_size = 10000
batch_size = 64
padded_shapes=tf.compat.v1.data.get_output_shapes(train_dataset)

padded_shapes_test=tf.compat.v1.data.get_output_shapes(test_dataset)

print(padded_shapes)

print(padded_shapes_test)

train_dataset = train_dataset.shuffle(buffer_size)
 # padded_batch是每批数据分别做padding
train_dataset = train_dataset.padded_batch(batch_size,padded_shapes) 

test_dataset = test_dataset.padded_batch(batch_size,padded_shapes_test)

(TensorShape([None, None]), TensorShape([None]))
(TensorShape([None, None]), TensorShape([None]))


In [32]:
vocab_size = tokenizer.vocab_size
embedding_dim = 16
batch_size = 512

bi_rnn_model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim),
    keras.layers.Bidirectional(
        keras.layers.LSTM(
            units = 32, return_sequences = False)),
    keras.layers.Dense(32, activation = 'relu'),
    keras.layers.Dense(1, activation='sigmoid'),
])

bi_rnn_model.summary()
bi_rnn_model.compile(optimizer = 'adam',
                     loss = 'binary_crossentropy',
                     metrics = ['accuracy'])


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, None, 16)          130960    
                                                                 
 bidirectional_4 (Bidirecti  (None, 64)                12544     
 onal)                                                           
                                                                 
 dense_8 (Dense)             (None, 32)                2080      
                                                                 
 dense_9 (Dense)             (None, 1)                 33        
                                                                 
Total params: 145617 (568.82 KB)
Trainable params: 145617 (568.82 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [33]:
history = bi_rnn_model.fit(
    train_dataset,
    #因为之前30次过拟合，这里改为10次
    epochs = 10,
    validation_data = test_dataset)

Epoch 1/10


ValueError: in user code:

    File "/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "/opt/conda/lib/python3.10/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/opt/conda/lib/python3.10/site-packages/keras/src/engine/input_spec.py", line 235, in assert_input_compatibility
        raise ValueError(

    ValueError: Ex

In [29]:
def plot_learning_curves(history,label,epochs,min_value,max_value):
    data = {}
    data[label] = history.history[label]
    data['val_'+label] = history.history['val_+label']
    pd.DataFrame(data).plot(figsize = (8,5))
    plt.grid(True)
    plt.axis([0,epochs,min_value,max_value])
    plt.show()
   
plot_learning_curves(history,'accuracy',10,0,1)
plot_learning_curves(history,'loss',10,0,1)
 
    

NameError: name 'history' is not defined